In [1]:
import pandas as pd; pd.set_option('display.max_columns', None)
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, precision_score, roc_auc_score, accuracy_score, f1_score

from lightgbm import LGBMClassifier

### Divorce Predictors data set

https://archive.ics.uci.edu/ml/datasets/Divorce+Predictors+data+set#

Набор состоит из 54 признаков, каждый признак принимает значение от 0 до 4.

In [8]:
df = pd.read_csv("divorce.csv", sep=";")
df.head(3)

,Atr1,Atr2,Atr3,Atr4,Atr5,Atr6,Atr7,Atr8,Atr9,Atr10,Atr11,Atr12,Atr13,Atr14,Atr15,Atr16,Atr17,Atr18,Atr19,Atr20,Atr21,Atr22,Atr23,Atr24,Atr25,Atr26,Atr27,Atr28,Atr29,Atr30,Atr31,Atr32,Atr33,Atr34,Atr35,Atr36,Atr37,Atr38,Atr39,Atr40,Atr41,Atr42,Atr43,Atr44,Atr45,Atr46,Atr47,Atr48,Atr49,Atr50,Atr51,Atr52,Atr53,Atr54,Class
0,2,2,4,1,0,0,0,0,0,0,1,0,1,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,2,1,2,0,1,2,1,3,3,2,1,1,2,3,2,1,3,3,3,2,3,2,1,1
1,4,4,4,4,4,0,0,4,4,4,4,3,4,0,4,4,4,4,3,2,1,1,0,2,2,1,2,0,1,1,0,4,2,3,0,2,3,4,2,4,2,2,3,4,2,2,2,3,4,4,4,4,2,2,1
2,2,2,2,2,1,3,2,1,1,2,3,4,2,3,3,3,3,3,3,2,1,0,1,2,2,2,2,2,3,2,3,3,1,1,1,1,2,1,3,3,3,3,2,3,2,3,2,3,1,1,1,2,2,2,1


In [9]:
# Соотношение классов

df.iloc[:, -1].value_counts()

0    86
1    84
Name: Class, dtype: int64

In [10]:
# Прорка пропущенных значений

df.isna().sum().sum()

0

Набор данных практически идеально сбалансирован, пропущенные значения отсутствуют.

Необходимости как-либо обрабатывать данные нет.

In [11]:
# Разделение на тренировочную и тестовую выборки

X = df.iloc[:,:-1]
y = df.iloc[:,-1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=17)

In [12]:
# Обучение модели и получение предсказания

model = LGBMClassifier(random_state=17)

model.fit(X_train, y_train)
y_predict = model.predict(X_test)

In [13]:
# Функция оценки качества 

def evaluate_results(y_test, y_predict):
    print('Classification results:')
    f1 = f1_score(y_test, y_predict)
    print("f1: %.2f%%" % (f1 * 100.0)) 
    roc = roc_auc_score(y_test, y_predict)
    print("roc: %.2f%%" % (roc * 100.0)) 
    rec = recall_score(y_test, y_predict, average='binary')
    print("recall: %.2f%%" % (rec * 100.0)) 
    prc = precision_score(y_test, y_predict, average='binary')
    print("precision: %.2f%%" % (prc * 100.0)) 
    metrics = (f1, roc, rec, prc)
    
    return metrics

In [14]:
# Оценка полученных результатов
    
results = evaluate_results(y_test, y_predict)
results

Classification results:
f1: 91.89%
roc: 92.50%
recall: 85.00%
precision: 100.00%


(0.9189189189189189, 0.925, 0.85, 1.0)

### PU learning

#### P = 10%

In [15]:
# Выборка из 10% позитивных значений

mod_df = df.copy()

pos_ind = np.where(mod_df.iloc[:,-1].values == 1)[0]

np.random.shuffle(pos_ind)

pos_sample_len = int(np.ceil(0.1 * len(pos_ind)))

pos_sample = pos_ind[:pos_sample_len]

In [16]:
# Добавим столбец новой целевой переменной class_test, где 1 - выбранные ранее позитивные значения (P), -1 - остальные (U)

mod_df['class_test'] = -1
mod_df.loc[pos_sample,'class_test'] = 1
print('target variable:\n', mod_df.iloc[:,-1].value_counts())

target variable:
 -1    161
 1      9
Name: class_test, dtype: int64


In [17]:
# random negative sampling

mod_df = mod_df.sample(frac=1)
neg_sample = mod_df[mod_df['class_test']==-1][:len(mod_df[mod_df['class_test']==1])]
sample_test = mod_df[mod_df['class_test']==-1][len(mod_df[mod_df['class_test']==1]):]
pos_sample = mod_df[mod_df['class_test']==1]
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

In [18]:
# Обучение модели и получение предсказания

model = LGBMClassifier(random_state=17)

model.fit(sample_train.iloc[:,:-2].values, 
          sample_train.iloc[:,-2].values)
y_predict = model.predict(sample_test.iloc[:,:-2].values)

In [19]:
# Оценка полученных результатов

results_10 = evaluate_results(sample_test.iloc[:,-2].values, y_predict)
results_10 

Classification results:
f1: 63.68%
roc: 50.00%
recall: 100.00%
precision: 46.71%


(0.6367713004484306, 0.5, 1.0, 0.46710526315789475)

#### P = 15%

In [20]:
# Выборка из 15% позитивных значений

mod_df = df.copy()

pos_ind = np.where(mod_df.iloc[:,-1].values == 1)[0]

np.random.shuffle(pos_ind)

pos_sample_len = int(np.ceil(0.15 * len(pos_ind)))

pos_sample = pos_ind[:pos_sample_len]

In [21]:
# Добавим столбец новой целевой переменной class_test, где 1 - выбранные ранее позитивные значения (P), -1 - остальные (U)

mod_df['class_test'] = -1
mod_df.loc[pos_sample,'class_test'] = 1
print('target variable:\n', mod_df.iloc[:,-1].value_counts())

target variable:
 -1    157
 1     13
Name: class_test, dtype: int64


In [22]:
# random negative sampling

mod_df = mod_df.sample(frac=1)
neg_sample = mod_df[mod_df['class_test']==-1][:len(mod_df[mod_df['class_test']==1])]
sample_test = mod_df[mod_df['class_test']==-1][len(mod_df[mod_df['class_test']==1]):]
pos_sample = mod_df[mod_df['class_test']==1]
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

In [23]:
# Обучение модели и получение предсказания

model = LGBMClassifier(random_state=17)

model.fit(sample_train.iloc[:,:-2].values, 
          sample_train.iloc[:,-2].values)
y_predict = model.predict(sample_test.iloc[:,:-2].values)

In [24]:
# Оценка полученных результатов

results_15 = evaluate_results(sample_test.iloc[:,-2].values, y_predict)
results_15 

Classification results:
f1: 64.79%
roc: 50.00%
recall: 100.00%
precision: 47.92%


(0.647887323943662, 0.5, 1.0, 0.4791666666666667)

#### P = 20%

In [25]:
# Выборка из 20% позитивных значений

mod_df = df.copy()

pos_ind = np.where(mod_df.iloc[:,-1].values == 1)[0]

np.random.shuffle(pos_ind)

pos_sample_len = int(np.ceil(0.20 * len(pos_ind)))

pos_sample = pos_ind[:pos_sample_len]

In [26]:
# Добавим столбец новой целевой переменной class_test, где 1 - выбранные ранее позитивные значения (P), -1 - остальные (U)

mod_df['class_test'] = -1
mod_df.loc[pos_sample,'class_test'] = 1
print('target variable:\n', mod_df.iloc[:,-1].value_counts())

target variable:
 -1    153
 1     17
Name: class_test, dtype: int64


In [27]:
# random negative sampling

mod_df = mod_df.sample(frac=1)
neg_sample = mod_df[mod_df['class_test']==-1][:len(mod_df[mod_df['class_test']==1])]
sample_test = mod_df[mod_df['class_test']==-1][len(mod_df[mod_df['class_test']==1]):]
pos_sample = mod_df[mod_df['class_test']==1]
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

In [28]:
# Обучение модели и получение предсказания

model = LGBMClassifier(random_state=17)

model.fit(sample_train.iloc[:,:-2].values, 
          sample_train.iloc[:,-2].values)
y_predict = model.predict(sample_test.iloc[:,:-2].values)

In [29]:
# Оценка полученных результатов

results_20 = evaluate_results(sample_test.iloc[:,-2].values, y_predict)
results_20 

Classification results:
f1: 60.51%
roc: 50.00%
recall: 100.00%
precision: 43.38%


(0.6051282051282051, 0.5, 1.0, 0.4338235294117647)

#### P = 25%

In [30]:
# Выборка из 25% позитивных значений

mod_df = df.copy()

pos_ind = np.where(mod_df.iloc[:,-1].values == 1)[0]

np.random.shuffle(pos_ind)

pos_sample_len = int(np.ceil(0.25 * len(pos_ind)))

pos_sample = pos_ind[:pos_sample_len]

In [31]:
# Добавим столбец новой целевой переменной class_test, где 1 - выбранные ранее позитивные значения (P), -1 - остальные (U)

mod_df['class_test'] = -1
mod_df.loc[pos_sample,'class_test'] = 1
print('target variable:\n', mod_df.iloc[:,-1].value_counts())

target variable:
 -1    149
 1     21
Name: class_test, dtype: int64


In [32]:
# random negative sampling

mod_df = mod_df.sample(frac=1)
neg_sample = mod_df[mod_df['class_test']==-1][:len(mod_df[mod_df['class_test']==1])]
sample_test = mod_df[mod_df['class_test']==-1][len(mod_df[mod_df['class_test']==1]):]
pos_sample = mod_df[mod_df['class_test']==1]
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

In [33]:
# Обучение модели и получение предсказания

model = LGBMClassifier(random_state=17)

model.fit(sample_train.iloc[:,:-2].values, 
          sample_train.iloc[:,-2].values)
y_predict = model.predict(sample_test.iloc[:,:-2].values)

In [34]:
# Оценка полученных результатов

results_25 = evaluate_results(sample_test.iloc[:,-2].values, y_predict)
results_25 

Classification results:
f1: 56.98%
roc: 50.00%
recall: 100.00%
precision: 39.84%


(0.5698324022346368, 0.5, 1.0, 0.3984375)

#### P = 30%

In [35]:
# Выборка из 30% позитивных значений

mod_df = df.copy()

pos_ind = np.where(mod_df.iloc[:,-1].values == 1)[0]

np.random.shuffle(pos_ind)

pos_sample_len = int(np.ceil(0.3 * len(pos_ind)))

pos_sample = pos_ind[:pos_sample_len]

In [36]:
# Добавим столбец новой целевой переменной class_test, где 1 - выбранные ранее позитивные значения (P), -1 - остальные (U)

mod_df['class_test'] = -1
mod_df.loc[pos_sample,'class_test'] = 1
print('target variable:\n', mod_df.iloc[:,-1].value_counts())

target variable:
 -1    144
 1     26
Name: class_test, dtype: int64


In [37]:
# random negative sampling

mod_df = mod_df.sample(frac=1)
neg_sample = mod_df[mod_df['class_test']==-1][:len(mod_df[mod_df['class_test']==1])]
sample_test = mod_df[mod_df['class_test']==-1][len(mod_df[mod_df['class_test']==1]):]
pos_sample = mod_df[mod_df['class_test']==1]
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

In [38]:
# Обучение модели и получение предсказания

model = LGBMClassifier(random_state=17)

model.fit(sample_train.iloc[:,:-2].values, 
          sample_train.iloc[:,-2].values)
y_predict = model.predict(sample_test.iloc[:,:-2].values)

In [39]:
# Оценка полученных результатов

results_30 = evaluate_results(sample_test.iloc[:,-2].values, y_predict)
results_30

Classification results:
f1: 95.92%
roc: 96.82%
recall: 97.92%
precision: 94.00%


(0.9591836734693877, 0.9681547619047619, 0.9791666666666666, 0.94)

#### Сравнение результатов

In [40]:
all_metrics = (results, results_10, results_15, results_20, results_25, results_30)

In [41]:
table = pd.DataFrame(list(all_metrics), 
        columns=["f-score", "roc-auc", "recall", "precision"], 
        index=["Base", "P = 10%", "P = 15%", "P = 20%", "P = 25%", "P = 30%"])

In [42]:
table

,f-score,roc-auc,recall,precision
Base,0.918919,0.925000,0.850000,1.000000
P = 10%,0.636771,0.500000,1.000000,0.467105
P = 15%,0.647887,0.500000,1.000000,0.479167
P = 20%,0.605128,0.500000,1.000000,0.433824
P = 25%,0.569832,0.500000,1.000000,0.398438
P = 30%,0.959184,0.968155,0.979167,0.940000


Вывод: Значительный прирост в качестве достигается при P = 30%